In [ ]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install numpy==1.18.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 30.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: Could not build wheels for numpy, which is required to install pyproject.toml-based projects


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pprint
import pandas as pd
import tempfile
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [ ]:
class Config: 
    
    embedding_dimension = 32
    
    model_path = "model.tf"
    
config = Config()

In [ ]:
cred = credentials.Certificate("/content/serviceAccountKey.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [ ]:
users = db.collection('users').get()
skills = db.collection('skills').get()
userData = []
for doc in users:
    data = doc.to_dict()
    userData.append(data)

profileData = []
for doc in users:
    data = doc.to_dict()
    profileData.append(data['profile'])

skillData = []
for doc in skills:
    data = doc.to_dict()
    skillData.append(data)


In [ ]:
userDf = pd.DataFrame(userData, columns=['uid', 'email'])
profileDf = pd.DataFrame(profileData, columns=['displayName', 'skills', 'interests'])
mergedDf = pd.merge(userDf, profileDf, left_index=True, right_index=True)

resultUser = mergedDf[['uid', 'email', 'displayName', 'skills', 'interests']]
resultUser

,uid,email,displayName,skills,interests
0,06yJpLuZ79Dbzyky0TQL,dlardez18@virginia.edu,Daren Lardez,"[{'name': 'Go', 'uid': 'Iz8pR1M5inaesq31fxQj'}...",[{'name': 'DevOps Engineer'}]
1,0CZfmWDndS6xjVv6o8j3,cpiolli1j@marketwatch.com,Carter Piolli,"[{'name': 'AWS'}, {'name': 'Ensemble Methods',...",[{'name': 'Web Analyst'}]
2,0M4QzQ4i5v3jFm0atItg,achrestienae@t-online.de,Alf Chrestien,"[{'uid': '5HX1httaNQ1e8mQhkiFc', 'name': 'Type...",[{'name': 'Big Data Specialist'}]
3,0Ot2WxvdSFpsKEI7ZJGx,jkleiserb@pbs.org,Jakie Kleiser,"[{'uid': 'lyOAoRMzwvDllSYMrLKH', 'name': 'Java...",[{'name': 'Enterprise System Integration Speci...
4,0RSTq7aC8ygoQxFrBN3A,tmullanea9@addtoany.com,Torry Mullane,"[{'uid': 'LU9jd5Wa1gYZKMH3iNsl', 'name': 'Reac...",[{'name': 'API Developer Specialist'}]
...,...,...,...,...,...
496,ygDFgGFFUk9Ay64ZNNBH,bforsdickecp@apple.com,Beitris Forsdicke,"[{'name': 'jQuery', 'uid': 'oPxmAOjOcWQPmO1J5m...",[{'name': 'Web Designer'}]
497,ykht9Jt6mCEk1TSpszKX,ydorant2y@comsenz.com,Yank Dorant,"[{'name': 'Scala', 'uid': '7o63SGr7UC7KShYh0jM...",[{'name': 'Android Technical developer'}]
498,z7CKDLeIahP4PwgZXYEm,cliversley4f@acquirethisname.com,Clarita Liversley,"[{'uid': 'vY6pmYvvcumYsDfbCBeA', 'name': 'Kotl...",[{'name': 'IT Capacity Planning Specialist'}]
499,zGY1ZQzBMWhiPgXwBHTt,cmcmurrayabg@typepad.com,Charmain McMurraya,"[{'name': 'JavaScript', 'uid': 'dprmLXoEiniNXi...",[{'name': 'Flutter Developer'}]


In [ ]:
userDf[['uid','email']]

,uid,email
0,06yJpLuZ79Dbzyky0TQL,dlardez18@virginia.edu
1,0CZfmWDndS6xjVv6o8j3,cpiolli1j@marketwatch.com
2,0M4QzQ4i5v3jFm0atItg,achrestienae@t-online.de
3,0Ot2WxvdSFpsKEI7ZJGx,jkleiserb@pbs.org
4,0RSTq7aC8ygoQxFrBN3A,tmullanea9@addtoany.com
...,...,...
496,ygDFgGFFUk9Ay64ZNNBH,bforsdickecp@apple.com
497,ykht9Jt6mCEk1TSpszKX,ydorant2y@comsenz.com
498,z7CKDLeIahP4PwgZXYEm,cliversley4f@acquirethisname.com
499,zGY1ZQzBMWhiPgXwBHTt,cmcmurrayabg@typepad.com


In [ ]:
resultUser['skills'] = resultUser['skills'].apply(lambda skill_list: ', '.join([skill_dict['name'] for skill_dict in skill_list]))
resultUser['interests'] = resultUser['interests'].apply(lambda interest_list:  ', '.join([interest_dict['name'] for interest_dict in interest_list]))

user = pd.DataFrame(resultUser)

user

,uid,email,displayName,skills,interests
0,06yJpLuZ79Dbzyky0TQL,dlardez18@virginia.edu,Daren Lardez,"Go, Usability Testing, IBM Cloud, React Native...",DevOps Engineer
1,0CZfmWDndS6xjVv6o8j3,cpiolli1j@marketwatch.com,Carter Piolli,"AWS, Ensemble Methods, React Native, IBM Cloud",Web Analyst
2,0M4QzQ4i5v3jFm0atItg,achrestienae@t-online.de,Alf Chrestien,"TypeScript, Hive, Clustering, vulnerability as...",Big Data Specialist
3,0Ot2WxvdSFpsKEI7ZJGx,jkleiserb@pbs.org,Jakie Kleiser,"Java, Naive Bayes, Objective-C, Oracle Cloud",Enterprise System Integration Specialist
4,0RSTq7aC8ygoQxFrBN3A,tmullanea9@addtoany.com,Torry Mullane,"React, Kafka, Decision Trees, cloud security",API Developer Specialist
...,...,...,...,...,...
496,ygDFgGFFUk9Ay64ZNNBH,bforsdickecp@apple.com,Beitris Forsdicke,"jQuery, Pig, Random Forest, network security",Web Designer
497,ykht9Jt6mCEk1TSpszKX,ydorant2y@comsenz.com,Yank Dorant,"Scala, Visual Design, DigitalOcean, iOS Develo...",Android Technical developer
498,z7CKDLeIahP4PwgZXYEm,cliversley4f@acquirethisname.com,Clarita Liversley,"Kotlin, Prototyping, Red Hat Cloud, Flutter De...",IT Capacity Planning Specialist
499,zGY1ZQzBMWhiPgXwBHTt,cmcmurrayabg@typepad.com,Charmain McMurraya,"JavaScript, Storm, Decision Trees, identity an...",Flutter Developer


In [ ]:
skill = pd.DataFrame(skillData)
skill

,uid,name
0,0sEFIEumkhfajK4SPZ3e,3D Modeling
1,1i9C101c8kpDnFdzCkUo,Spark
2,2cNqonnZ6Xr59H7VYy9O,Rust
3,2xiEcPCDINE7Axd6gIww,security operations
4,3lZL8HMAEsws5MHWIdvu,Salesforce Cloud
...,...,...
112,ylZBy3GmRz1hqK6Mz4Wh,Networking
113,z3vEA0zwEcUBqfdujxjV,Decision Trees
114,zX9WB9armnM8xqh1TiV3,Ensemble Methods
115,zmyXaiGJ4x8TpyOdNBSt,application security


In [ ]:
user = tf.data.Dataset.from_tensor_slices(dict(user))
skill = tf.data.Dataset.from_tensor_slices(dict(skill))

In [ ]:
# get the first rows of the movies dataset
for m in skill.take(5):
  print(m)

{'uid': <tf.Tensor: shape=(), dtype=string, numpy=b'0sEFIEumkhfajK4SPZ3e'>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'3D Modeling'>}
{'uid': <tf.Tensor: shape=(), dtype=string, numpy=b'1i9C101c8kpDnFdzCkUo'>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Spark'>}
{'uid': <tf.Tensor: shape=(), dtype=string, numpy=b'2cNqonnZ6Xr59H7VYy9O'>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Rust'>}
{'uid': <tf.Tensor: shape=(), dtype=string, numpy=b'2xiEcPCDINE7Axd6gIww'>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'security operations'>}
{'uid': <tf.Tensor: shape=(), dtype=string, numpy=b'3lZL8HMAEsws5MHWIdvu'>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Salesforce Cloud'>}


In [ ]:
user = user.map(lambda x: {
    "uid": x["uid"],
    "displayName": x["displayName"],
    "skills": x["skills"],
    "interests": x["interests"]
})
skill = skill.map(lambda x: {
    "uid": x["uid"],
    "name": x["name"]
}) 

In [ ]:
user_array = []
skill_array = []
for data in user:
    user_array.append(data['uid'].numpy().decode())
for data in skill:
    skill_array.append(data['name'].numpy().decode())


#skills_array = tf.squeeze(skills).numpy().astype(str).tolist()
#jobs_array = tf.squeeze(jobs).numpy().astype(str).tolist()

user_ids_vocabulary = tf.keras.layers.StringLookup()
user_ids_vocabulary.adapt(user_array)

skill_titles_vocabulary = tf.keras.layers.StringLookup()
skill_titles_vocabulary.adapt(skill_array)


In [ ]:
for data in skill:
  print(data['name'])

tf.Tensor(b'3D Modeling', shape=(), dtype=string)
tf.Tensor(b'Spark', shape=(), dtype=string)
tf.Tensor(b'Rust', shape=(), dtype=string)
tf.Tensor(b'security operations', shape=(), dtype=string)
tf.Tensor(b'Salesforce Cloud', shape=(), dtype=string)
tf.Tensor(b'Laravel', shape=(), dtype=string)
tf.Tensor(b'Linux', shape=(), dtype=string)
tf.Tensor(b'User Research', shape=(), dtype=string)
tf.Tensor(b'compliance', shape=(), dtype=string)
tf.Tensor(b'TypeScript', shape=(), dtype=string)
tf.Tensor(b'AWS', shape=(), dtype=string)
tf.Tensor(b'cybersecurity awareness', shape=(), dtype=string)
tf.Tensor(b'Sales', shape=(), dtype=string)
tf.Tensor(b'Regression', shape=(), dtype=string)
tf.Tensor(b'Scala', shape=(), dtype=string)
tf.Tensor(b'malware analysis', shape=(), dtype=string)
tf.Tensor(b'Deep Learning', shape=(), dtype=string)
tf.Tensor(b'Negotiation', shape=(), dtype=string)
tf.Tensor(b'Storm', shape=(), dtype=string)
tf.Tensor(b'network security', shape=(), dtype=string)
tf.Tensor(b'C

In [ ]:
class JobLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.
 
  def __init__(
      self,
      user_model: tf.keras.Model,
      skill_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()
 
    # Set up user and movie representations.
    self.user_model = user_model
    self.skill_model = skill_model

 
    # Set up a retrieval task.
    self.task = task
 
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
 
    user_embeddings = self.user_model(features["uid"])
    skill_embeddings = self.skill_model(features["name"])

 
    return self.task(user_embeddings, skill_embeddings)

In [ ]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    tf.keras.Input(shape=[], dtype=tf.string, name='uid'),
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
skill_model = tf.keras.Sequential([
    tf.keras.Input(shape=[], dtype=tf.string, name='name'),
    skill_titles_vocabulary,
    tf.keras.layers.Embedding(skill_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        skill.batch(128).map(skill_model)
    )
)

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['uid'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


In [ ]:
# Create a retrieval model.
model = JobLensModel(user_model, skill_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
 
# Train for 3 epochs.
model.fit(skill.batch(4096), epochs=50)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0085 - factorized_top_k/top_5_categorical_accuracy: 0.0427 - factorized_top_k/top_10_categorical_accuracy: 0.0855 - factorized_top_k/top_50_categorical_accuracy: 0.4274 - factorized_top_k/top_100_categorical_accuracy: 0.8462 - loss: 557.1771 - regularization_loss: 0.0000e+00 - total_loss: 557.1771
Epoch 2/50
1/1 [==============================] - 0s 95ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0085 - factorized_top_k/top_5_categorical_accuracy: 0.0513 - factorized_top_k/top_10_categorical_accuracy: 0.1026 - factorized_top_k/top_50_categorical_accuracy: 0.4359 - factorized_top_k/top_100_categorical_accuracy: 0.8291 - loss: 557.1756 - regularization_loss: 0.0000e+00 - total_loss: 557.1756
Epoch 3/50
1/1 [==============================] - 0s 121ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0085 - factorized_top_k/top_5_categorical_accuracy: 0.0513 - factor

In [ ]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.skill_model)
index.index_from_dataset(
    user.batch(100).map(lambda x: (x['uid'], model.user_model(x['uid'])))
)

# Get some recommendations.
ids = tf.constant(["5HX1httaNQ1e8mQhkiFc"])
_, titles = index(tf.constant(ids))
for i in range(len(ids)):
    print(f"Recommendations for user id {ids[i].numpy()}: {titles[i].numpy()}")

Recommendations for user id b'5HX1httaNQ1e8mQhkiFc': [b'4Gderx6kOPf3TOJ58ZLY' b'kKz0m7MDxY172hYb1ONq' b'CAQ5mVx23MVmqMdewRxN'
 b'CVdnuxrrzWiTQp9DJ3Nb' b'ovEJv4IfAwjtxvu5epBY' b'EPE380kTQV4Th8jMjZzP'
 b'lZ3HAC3D76WG4JHHXCuR' b'xzo0j6ji6Ft3mDykHAhr' b'b7R9HCVnvHQt8FjRlvIm'
 b'IxuhqjWNeeh24w0viJ18']


In [ ]:
def recommend(id):
    # Use brute-force search to set up retrieval using the trained representations.
    index = tfrs.layers.factorized_top_k.BruteForce(model.skill_model)
    index.index_from_dataset(
        user.batch(100).map(lambda x: (x['uid'], model.user_model(x['uid'])))
    )
    
    # Get recommendations for the given user ID.
    ids = tf.constant([id])
    _, titles = index(tf.constant(ids))
    
    # Print the recommendations.
    recommendations = titles.numpy().tolist()
    return recommendations
  

In [ ]:
recommend('5HX1httaNQ1e8mQhkiFc')

[[b'4Gderx6kOPf3TOJ58ZLY',
  b'kKz0m7MDxY172hYb1ONq',
  b'CAQ5mVx23MVmqMdewRxN',
  b'CVdnuxrrzWiTQp9DJ3Nb',
  b'ovEJv4IfAwjtxvu5epBY',
  b'EPE380kTQV4Th8jMjZzP',
  b'lZ3HAC3D76WG4JHHXCuR',
  b'xzo0j6ji6Ft3mDykHAhr',
  b'b7R9HCVnvHQt8FjRlvIm',
  b'IxuhqjWNeeh24w0viJ18']]